In [8]:
import cv2
import numpy as np
from scipy.fftpack import fft2, ifft2
from skimage import exposure
import pywt
import os

In [96]:
def wavelet_transform(image):
    # Apply wavelet transformation
    coeffs = pywt.dwt2(image, 'bior1.3')
    cA, (cH, cV, cD) = coeffs
    return cA

def fft_transform(image):
    # Apply FFT transformation
    fft_result = fft2(image)
    return fft_result

def log_prob_transform(image):
    # Apply log probability transform
    log_prob_result = np.log(1 + np.abs(image))
    return log_prob_result

def high_pass_filtering(image):
    # Apply high-pass filtering
    rows, cols = image.shape
    crow, ccol = rows // 2, cols // 2

    # Create a mask for high-pass filtering
    mask = np.ones((rows, cols), np.uint8)
    r = 30  # Adjust this parameter for the size of the high-pass filter
    center = [crow, ccol]
    x, y = np.ogrid[:rows, :cols]
    mask_area = (x - center[0]) ** 2 + (y - center[1]) ** 2 <= r*r
    mask[mask_area] = 0

    # Apply high-pass filter
    image_filtered = image * mask
    return image_filtered

def get_feature_vector(img):
    # Step 1: Apply wavelet transformation
    trans = wavelet_transform(img)

    # Step 2: Apply FFT to output 1
    trans = fft_transform(trans)

    # Step 3: Apply log probability transform on output 2
    trans = log_prob_transform(trans)

    # Step 4: High-pass filtering on output 3
    hpf = high_pass_filtering(trans)

    # Step 5: Apply FFT to output 4
    fft = fft_transform(hpf)

    # Concatenate output 4 and output 5
    output = np.concatenate((hpf, np.abs(fft)), axis=None)

    # Extract feature vector from concatenated output
    output = output.flatten()
    
    return output

def biohashing(img_vec, M, m, k, thresh):
    # Step 1: Generate pseudo-random vectors {ri} based on user's seed k
    np.random.seed(k)
    r = np.random.rand(m, M)
    
    # Step 2: Apply Gram-Schmidt process to transform r into an orthonormal set of matrices
    r_orthogonal = np.array([r[0]])
    for i in range(1, m):
        ri = r[i]
        for j in range(i):
            ri -= np.dot(r[i], r_orthogonal[j]) * r_orthogonal[j]
        ri /= np.linalg.norm(ri)
        r_orthogonal = np.vstack([r_orthogonal, ri])
    
    # Step 3: Compute ⟨img_vec, r⊥i⟩ for each r⊥i
    inner_products = np.dot(img_vec, r_orthogonal.T)
    
    # Step 4: Compute m bits BioHash {bi}
    bi = []
    for i in range(m):
        if inner_products[i] <= thresh:
            bi.append(0)
        else:
            bi.append(1)
    return bi

def binatodeci(binary):
    return sum(val*(2**idx) for idx, val in enumerate(reversed(binary)))

In [152]:
m = 16 # Replace with the desired length of the protected template
thresh = 0.4  # Replace with the preset threshold

# sigmas = [3, 4, 5, 6, 7, 8, 9, 10] Others dont give unqiue biohash, so delted from database
sigmas = [4, 8, 9]
sigma_nums = []

# All feature vectors with sigma values 3... 10
for sigma in sigmas:
    dataset_path = f'/home/manjunathjoshi/202211050/major_project/datasets/LR-VERA-fingervein-max-curvature-images-sigma-{sigma}'
    # print(f'*******************************************************************************************************************')
    # print(f'******************************************************** {sigma} ********************************************************')
    # print(f'*******************************************************************************************************************')
    
    biohash_nums = []
    k = 202211000 # Replace with the user's seed
    
    # User_1.... User_220
    for user_class in os.listdir(dataset_path):
        user_folder = os.path.join(dataset_path, user_class)

        k += 1
#         if k >= 202211006:
#             break

        for user_img in os.listdir(user_folder):
            img_path = os.path.join(dataset_path, user_class, user_img)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            feature_vec = get_feature_vector(image.copy())
            M = len(feature_vec)

            binary_temp = biohashing(feature_vec.copy(), M, m, k, thresh)
            decimal_temp = binatodeci(binary_temp)
            
            # print(user_class, user_img, k, decimal_temp)
            # print()
        biohash_nums.append(decimal_temp)
    sigma_nums.append(biohash_nums)

In [153]:
print(sigma_nums[0])

[37144, 39592, 37392, 48523, 40216, 47649, 40212, 44659, 47484, 46078, 37135, 45468, 48634, 34605, 43792, 39263, 35279, 46044, 37371, 35313, 43314, 32889, 42321, 47174, 41045, 39762, 42035, 40030, 47797, 44297, 34364, 45783, 49026, 33256, 42377, 33405, 42871, 44859, 42135, 48633, 36030, 46358, 46468, 46250, 42780, 45818, 38544, 39771, 45677, 34055, 33127, 45129, 35604, 44880, 41859, 35930, 41250, 48821, 46969, 45189, 35144, 41845, 37724, 35379, 45950, 40800, 32837, 35449, 42156, 35898, 33589, 40434, 43499, 35368, 42219, 48097, 43085, 34065, 48963, 42385, 43482, 36851, 34846, 45955, 39191, 41228, 39938, 33398, 43784, 33829, 38123, 46662, 41965, 44062, 34419, 44957, 38051, 47868, 44960, 47225, 43186, 45250, 42688, 46093, 35096, 41860, 41097, 47074, 46102, 41295, 39870, 42101, 35132, 47901, 38299, 35256, 45156, 45287, 35562, 34625, 34648, 40919, 34559, 46230, 48459, 45461, 35020, 39171, 34986, 43013, 37647, 44068, 34233, 38807, 44602, 47570, 45804, 43512, 37338, 40280, 47630, 46722, 37575

In [154]:
len(sigma_nums[0])

220

In [156]:
from collections import Counter

for ind, input_list in enumerate(sigma_nums):
    
    items = Counter(input_list).keys()
    print(f'No of unique items in sigma {ind+3} list are:', len(items))

No of unique items in sigma 3 list are: 218
No of unique items in sigma 4 list are: 220
No of unique items in sigma 5 list are: 219
No of unique items in sigma 6 list are: 219
No of unique items in sigma 7 list are: 219
No of unique items in sigma 8 list are: 220
No of unique items in sigma 9 list are: 220
No of unique items in sigma 10 list are: 219


In [157]:
def save_numbers_to_notepad(numbers, file_path):
    try:
        with open(file_path, 'w') as file:
            for number in numbers:
                file.write(str(number) + '\n')
        print(f"Numbers saved to {file_path} successfully.")
    except Exception as e:
        print(f"Error: {e}")


        # Example usage:
numbers_list = [1, 2, 3, 4, 5]

# Specify the file path where you want to save the numbers
file_path = 'biohash_numbers.txt'

# Call the function to save numbers to the Notepad file
save_numbers_to_notepad(sigma_nums[5], file_path)

Numbers saved to biohash_numbers.txt successfully.
